# Practice SQL

Okay, our `SQL` feet are wet. We have seen how to load databases and how to query databases so let's step a little further into the `SQL` pool and query a larger database. In the previous notebooks, we were using a `SQLite` engine but `SQLite` is...well...lite. Instead, in this practice we are going to work with a `PostgreSQL` database, which has one of the most advanced open source `SQL` clients available. Before we begin, let's talk about the data.

### Twitter Data

We'll be working with "some" Twitter data in this practice. "Some" is in quotes only because we are working with over 250 million rows (250 with 6 ZEROS) and that is only "some" of what is being collected. This database has been collecting from 50 different US cities since from Jan 2017 until mid May 2017 for this particular copy of the database. 

--- 

Given the volume of data, this database server is a good introduction to some other database concepts we have yet to touch on. Again, `pandas` has some pretty neat tools for interacting with databases of all flavors. All we have to do is establish the connection and we can start making requests of the database.

In [1]:
import psycopg2
import pandas as pd

try:
    connect_str = "dbname='twitter' user='dsa_ro_user' host='pgsql.dsa.lan' password='readonly'"
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
except:
    print("Something went wrong...probably the wrong permissions")

`psycopg2` is a package that allows `Python` to connect to a `Postgres` database.

We didn't state this explicitly, but databases allow us to query other aspects of it other than tables. We can do something like write a query to get the table names in the database. Let's do that now so we can get an idea of what types of tables we will be working with.

In [2]:
statement = """SELECT table_name 
    FROM information_schema.tables
    WHERE table_schema = 'twitter'"""

pd.read_sql_query(statement,conn)

,table_name
0,mention
1,new_slur
2,pred_output2
3,pred_output1
4,tweet
5,url
6,job
7,hashtag


So we are working with a `tweet`, `hashtag`, `mention`, `url`, and `job` table, but that doesn't tell us a whole lot about the contents of any given table. At most, we can infer from the table names what type of data might be in them. But it would be better to know the attributes and perhaps what kind of data type they are. 

Again, we will be referencing the database's `information_schema` in order to pull this information out. Study the following query and compare it with the last.

In [3]:
statement = """                              
SELECT column_name, data_type, is_nullable
FROM information_schema.columns
WHERE table_name = 'hashtag';
"""

pd.read_sql_query(statement,conn)

,column_name,data_type,is_nullable
0,hashtag_id,bigint,NO
1,tweet_id,character varying,NO
2,text,character varying,NO
3,index_start,smallint,NO
4,index_end,smallint,NO
5,job_id,integer,NO
6,analysis_state,integer,YES


Now we know a bit about the contents of the of the `hashtag` table, which is good because we will be using it quite a bit in this practice. 

**Take a moment and contemplate the `CREATE TABLE` command that was used to create this table.**

Moving on, we can explore what type of attributes are in the `tweet` data?

**Activity 1:** Write a query to look at the `column_name`s, `data_type`s and `character_maximum_length`s of the `tweet` table.

In [4]:
# Your code for Activity 1 goes here
# --------------

statement = """                              
SELECT column_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name = 'tweet';
"""

pd.read_sql_query(statement,conn)

,column_name,data_type,character_maximum_length
0,tweet_id_str,character varying,20.0
1,job_id,integer,NaN
2,created_at,timestamp without time zone,NaN
3,text,text,NaN
4,from_user,character varying,45.0
5,from_user_id_str,character varying,20.0
6,from_user_name,character varying,45.0
7,from_user_fullname,text,NaN
8,from_user_created_at,timestamp without time zone,NaN
9,from_user_followers,integer,NaN


**Activity 2:** Now lets write a querry that would get **all** of the column characteristics (ie. `column_name`, `data_type`, etc.) for a single table, lets say the 'tweet' table.

In [5]:
# Your code for Activity 2 goes here
# --------------

statement = """                              
SELECT *
FROM information_schema.columns
WHERE table_name = 'tweet';
"""

pd.read_sql_query(statement,conn)

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,twitter,twitter,tweet,tweet_id_str,1,None,NO,character varying,20.0,8.000000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,twitter,twitter,tweet,job_id,2,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,twitter,twitter,tweet,created_at,3,None,NO,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,twitter,twitter,tweet,text,4,None,NO,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,twitter,twitter,tweet,from_user,5,None,NO,character varying,45.0,1.800000e+02,...,NO,None,None,None,None,None,NO,NEVER,None,YES
5,twitter,twitter,tweet,from_user_id_str,6,None,NO,character varying,20.0,8.000000e+01,...,NO,None,None,None,None,None,NO,NEVER,None,YES
6,twitter,twitter,tweet,from_user_name,7,None,NO,character varying,45.0,1.800000e+02,...,NO,None,None,None,None,None,NO,NEVER,None,YES
7,twitter,twitter,tweet,from_user_fullname,8,None,NO,text,NaN,1.073742e+09,...,NO,None,None,None,None,None,NO,NEVER,None,YES
8,twitter,twitter,tweet,from_user_created_at,9,None,NO,timestamp without time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
9,twitter,twitter,tweet,from_user_followers,10,None,NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES


That's enough surface level exploring for now. 
Let's get our hands dirty a bit. 
To do that, we have to dig ;) into the data. 

Oh! but before we dig in we need to cover the idea of `LIMIT`. 
There are several reasons to put a `limit` on the number of rows returned in a query. 
For example, if we just want to get an idea of what our data looks like, we might specify a `LIMIT` of `5`. 
This is similar to running `head(dataframe_name)` in `R` or `dataframe_name.head()` in `Python`. 
But the reason we are using it is more for performance reasons. 
Remember, this is a fairly large database, and not specifying a limit and trying to pull millions of rows of data into Pandas could slow the operation down to a pace that is beyond the limit of your patience. 

So let's introduce `LIMIT` right now.

```SQL
SELECT <column_names>
FROM <table_name>
LIMIT <numeber_of_rows>
```

In [6]:
statement = """
SELECT * 
FROM twitter.hashtag
LIMIT 1000
"""

pd.read_sql_query(statement,conn)

,hashtag_id,tweet_id,text,index_start,index_end,job_id,analysis_state
0,1,823609394529271808,activityprofessionalsweek,0,26,205,0
1,2,823609285519110146,ForCityAndClub,121,136,205,0
2,3,823608842441269249,job,5,9,205,0
3,4,823608842441269249,Nursing,96,104,205,0
4,5,823608842441269249,Fresno,105,112,205,0
...,...,...,...,...,...,...,...
995,996,823530149492695040,jobs4u,0,7,205,0
996,997,823530149492695040,jobs,8,13,205,0
997,998,823530149492695040,FRE,52,56,205,0
998,999,823530149492695040,fresno,57,64,205,0


#### Nested Queries

So **LIMIT**s employed for performance sake make for the appropriate time to cover nested queries. Nested queries are a good way to extract further information after performing some operation. Take the example below. We are trying to return a table that displays the unique hashtags in one column and how many times they occur in another. But remember, this database is large and we don't want to perform this over the entire table, so instead, we are going to perform this query over another query nested inside of it where we limit the number of rows to 1000. 

We can break this query apart from the inner part first to the outer part.

```SQL
SELECT text
FROM twitter.hashtag
LIMIT 1000
```

We know what this query is going to return. It's going to return a table of only the `text` column form the `hashtag` table, but only the first 1000 rows. We will nest this table inside of another query so that it is only performing an aggregation off of a subset (1000 rows) of the data:

```SQL
SELECT DISTINCT text, COUNT(*)
FROM( ...
...) AS t1
GROUP BY text
```

In this outer statement, we are wanting to know the unique terms as well as their counts for that 1000 rows we pull. The `AS t1` just names the nested query. Now we can take a look at what this actually returns.

In [7]:
statement = """
SELECT DISTINCT text, COUNT(*)
FROM(
    SELECT text
    FROM twitter.hashtag
    LIMIT 1000) AS t1
GROUP BY text
"""

pd.read_sql_query(statement,conn)

,text,count
0,Cowboy,1
1,paulmccartney,1
2,Artist,1
3,Hospice,1
4,Kingsburg,1
...,...,...
357,science,1
358,Healthcare,1
359,lit,1
360,s,4


Take a look at the column names of the returned data frame: `text` and `count`.  Right now there is no order and that can get a little annoying...


**Activity 3:** Order the query above by `count`. 


In [8]:
# Your code for Activity 3 goes here
# --------------

statement = """
SELECT DISTINCT text, COUNT(*)
FROM(
    SELECT text
    FROM twitter.hashtag
    LIMIT 1000) AS t1
GROUP BY text
ORDER BY count;
"""

pd.read_sql_query(statement,conn)

,text,count
0,Accounting,1
1,activityprofessionalsweek,1
2,ADogsPurpose,1
3,adverseeffects,1
4,agfax,1
...,...,...
357,ASMSG,24
358,CareerArc,31
359,Hiring,33
360,job,41


And of course we want to know the most popular hashtags by `count`.

**Activity 4:** Order the same query above from greatest to fewest count.

In [9]:
# Your code for Activity 4 goes here
# ----------------

statement = """
SELECT DISTINCT text, COUNT(*)
FROM(
    SELECT text
    FROM twitter.hashtag
    LIMIT 1000) AS t1
GROUP BY text
ORDER BY count DESC;
"""

pd.read_sql_query(statement,conn)

,text,count
0,Fresno,113
1,job,41
2,Hiring,33
3,CareerArc,31
4,ASMSG,24
...,...,...
357,المطاعم,1
358,اليمن,1
359,تعز,1
360,صنعاء,1


Many databases also give the user the ability to perform operations on the columns within the query. These operations are very helpful in removing data carpentry steps from later on in our analysis. 

You may have noticed that some hashtags are actually repeated twice (or thrice or more) despite using `DISTINCT` on the `text` column. Can you identify why this is? Well, it is because `DISTINCT` is case sensitive and some hashtags are capitalized while others aren't. Fortunately, there is an operation that we can perform on the `text` column to transform these values all to one case on the fly.

Take a look at the `LOWER()` operation...

In [10]:
statement = """
SELECT LOWER(text) 
FROM twitter.hashtag
LIMIT 1000
"""

pd.read_sql_query(statement,conn)

,lower
0,activityprofessionalsweek
1,forcityandclub
2,job
3,nursing
4,fresno
...,...
995,jobs4u
996,jobs
997,fre
998,fresno


**Activity 5:** Now, similar to the queries above, count the number of distinct hashtag `text`s but this time transform `text` to lower. Remember to put a `LIMIT` of 1000 in the nested query. Be sure to put the count in descending order.

*HINT*: `LOWER()` should go in the nested portion.

In [11]:
# Your code for Activity 5 goes here
# --------------

statement = """
SELECT DISTINCT text, COUNT(*)
FROM(
    SELECT lower(text) AS text
    FROM twitter.hashtag
    LIMIT 1000) AS t1
GROUP BY text
ORDER BY count DESC;
"""

pd.read_sql_query(statement,conn)

,text,count
0,fresno,134
1,job,55
2,hiring,47
3,hollyoaks,38
4,careerarc,33
...,...,...
335,المطاعم,1
336,اليمن,1
337,تعز,1
338,صنعاء,1


And there is so much more than just the `COUNT` of rows. You can also find the `AVERAGE` (arithmetic mean) of a numeric column. Keep in mind that the query below is `GROUP`ed `BY` `text`.

In [12]:
statement = """
SELECT DISTINCT text, AVG(index_start)
FROM(
    SELECT lower(text) AS text, index_start
    FROM twitter.hashtag
    LIMIT 1000) AS t1
GROUP BY text
ORDER BY avg DESC
"""

pd.read_sql_query(statement,conn)

,text,avg
0,uber,136.0
1,ne,135.0
2,care,134.0
3,salute,132.0
4,trump,132.0
...,...,...
335,politics,0.0
336,tgdtla,0.0
337,trusttheprocess,0.0
338,usanews,0.0


We're going to return to the world of joins now. There is one thing we should add about `JOIN`s that we didn't have the opportunity to discuss in the lab. It is possible to perform a full outer join without using the `JOIN` statement. All you have to do is specify is what columns to match from the two (or more) tables in the `WHERE` clause. 

Take the example below. We are wanting to return all rows from both tables where the tweet_id matches in both the `hashtag` and `tweet` tables. Keep in mind, the `tweet` table has the column named `tweet_id_str`.

In [13]:
statement = """
SELECT t.tweet_id_str, t.text, h.text
FROM twitter.tweet t, twitter.hashtag h
WHERE t.tweet_id_str = h.tweet_id
LIMIT 100;
"""

pd.read_sql_query(statement,conn)

,tweet_id_str,text,text
0,823609394529271808,#activityprofessionalsweek @ Healthcare Center...,activityprofessionalsweek
1,823609285519110146,RT @FresnoFuego: NEWS: The Hwy 99 Derby + @Sac...,ForCityAndClub
2,823608842441269249,This #job might be a great fit for you: Critic...,job
3,823608842441269249,This #job might be a great fit for you: Critic...,Nursing
4,823608842441269249,This #job might be a great fit for you: Critic...,Fresno
...,...,...,...
95,823603880420249600,Monday's are ruff. How do my @Fresno_State #Bu...,MondayMotivation
96,823603783305482241,County of Fresno is hiring a DEPUTY SHERIFF RE...,police
97,823603783305482241,County of Fresno is hiring a DEPUTY SHERIFF RE...,jobsearch
98,823603286297219073,Fresno City College Get your 10 FREE LYFT RIDE...,university
